
# Python API
Using pygn from https://github.com/cweichen/pygn

In [294]:
import sys

import numpy as np
import pandas as pd
import datetime
import time

# music api imports
import spotipy
import spotipy.util as util
import pygn
#import pylast

# import requests
#import requests
#import re
from time import sleep
import pickle

# import python files
import dbutils

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [297]:
clientID = '18404282-D215A8393036073C7D4008B567ADB278' # Enter your Client ID here
#userID = pygn.register(clientID)
userID= '91860226739687882-7862D6C2BEF5300543BC0BFE3BE6F36D'

In [298]:
def get_full_path():
    return "/Users/feelharmonic/Dropbox/Insight2018/SaReGaMood/IndianClassicalMusic/Week2/playlist_data"

In [299]:
metadata = pygn.search(clientID=clientID, userID=userID, artist='Kings Of Convenience', album='Riot On An Empty Street', track='Homesick')

The search function requires a clientID, userID, and at least one of either artist, album, or track to be specified.

In [300]:
metadata = pygn.search(clientID=clientID, userID=userID, artist='Jaga Jazzist', album='What We Must')
#print(metadata)

In [301]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print(f.__name__, 'took', end - start, 'seconds')
        return result
    return f_timer

In [302]:
# Setting up tokens and authorization

# Gracenote
clientid = '18404282-D215A8393036073C7D4008B567ADB278'
# userid = pygn.register(clientid)
userid = '40350305677161399-FBEEF011A5EE5144578CE45E3A2EC94D'



In [303]:
# Get track info from gracenote
# NOTE: Gracenote API has built-in fuzzy matching to artist and track.
from collections import defaultdict

def get_gn_multiple(search, dictionary, item):
    '''
    Helper function to get multiple items within Gracenote record
    '''
    #print("~~~~~~", "item = ", str(item), "\n", "searchitem = ", str(search[item]), "~~~~~~\n")
    #for i in search[item].iteritems():
    #    print("itemi[0] =", item + "_"+i[0])
    #    print("i1text =", i[1]['TEXT'])
    #    dictionary[item + '_' + i[0]] = i[1]['TEXT']
    for k, v in search[item].items():
        #print("k, v = ", k, v)
        dictionary[item + '_' + k] = v['TEXT']

def get_gn(artist, track):
    '''
    Gets artist and track information from Gracenote
    '''
    gn_dict = defaultdict(list)
    gn_info = pygn.search(clientid, userid, artist=artist, track=track)
    #print("gn_info = ", str(gn_info))

    gn_dict['gnid'] = gn_info['track_gnid']
#     print("gn_info = ", str(gn_info))
    #print("gn_dict = ", str(gn_dict), "======\n")

    # artist specific info
    # for a in ['artist_origin', 'artist_type', 'artist_era']:
    #    get_gn_multiple(gn_info, gn_dict, a)
    
    # track specific info
#     for s in ['genre', 'mood', 'tempo']: # can potentially drop 'tempo' since Spotify has already captured this
    for s in ['mood']:  
        #print("Getting moody stuff")
        get_gn_multiple(gn_info, gn_dict, s)      

#     print("gn_dict = ", str(gn_dict), "======\n")
    return dict(gn_dict)

In [304]:
@timefunc
def append_gn_df(df):
    '''
    Creates dataframe with artist and track information from Gracenote
    '''
    d = {}
    for index, row in df.iterrows():
        try:
            d[row['id']] = get_gn(row['artist_name'], row['name'])
        except:
            print("Some issue in append_gn_df")
            pass
    gn_df = pd.DataFrame(d).transpose().reset_index()
    gn_df = gn_df.rename(columns = {'index' : 'id'})
    return gn_df

In [305]:
@timefunc
def chunk_gn_df(df, start, stop, step=100):
    '''
    Gets Gracenote dataframe from dataframe with spotify IDs and merges them, indexing based on Spotify dataframe
    '''
    full_songs = pd.DataFrame()
    for i in range(start, stop, step):
        myslice = [x for x in range(i, i+step)]
        gn = append_gn_df(df.iloc[myslice])
        print("gn = ", str(gn), "==================")
        data = pd.merge(df, gn, on='id')
        full_songs = full_songs.append(data, ignore_index=True)
        sleep(.1)

In [306]:
con = dbutils.get_db_handle()
#querystr = "select id, name, artist_name from alltracks where name like \'%Tarana%\' OR name like \'%Tillana%\' OR name like \'%Thillana%\'"
querystr = "select id, name, artist_name from alltracks where name like \'%Alap%\' OR name like \'%Khayal%\' OR name like \'%Khyal%\'"
df = dbutils.query_db_translate_to_pandas(con, querystr)
# print(df.to_string())
dbutils.close_db(con)

In [275]:
test = df[:10].copy()
test_gn_df = append_gn_df(test)
test2 = df[:100].copy()
test_gn_df2 = append_gn_df(test2)

append_gn_df took 6.29734992980957 seconds
append_gn_df took 70.43506002426147 seconds


In [276]:
print(df.shape)

(1377, 3)


In [307]:
test3 = df[:80].copy()
print(test3.shape)
test = chunk_gn_df(test3, 0, test3.shape[0], 40)
# print(test3)

(80, 3)
append_gn_df took 28.357304096221924 seconds
gn =                          id                                        gnid  \
0   0KYXDtD08zE6EeeRQHJp1l   96995694-A3125A24F4F157CD43F10F33E3BD6435   
1   0OA8aB0Af365psj1jIKF8R    4951315-0AA9B2427BFEF9E5763A72676EB1ADDA   
2   0SrZTZKPplckh7ZIfJ1DTR  396822763-0AFEB1337DFB8137FC3CC5F6AA9E1C5D   
3   0UtucXLVbm7CNwS8O49zMJ  172181751-1D65913E472E661F01C40B25E1FBFAD4   
4   0W5FwN9zG4qyHpQYP0cAHC   81004917-8A8DCD1E864526B8BBB62A364CB63CB6   
5   0hewhn6vFioyvZJHKNn6ZS  142133950-4C17EB83B336AB33619204B8649946F6   
6   0n3inpHtzX9fGqx14Lrm5z  429990310-698EB8424A51B737764DE6528F8A6A69   
7   0rIRZgSzBQEwrsjK36mJ2O    5733348-6C8182FA80CFDA1CF36264AD67549B26   
8   0z5yfdMQtOxLDXlGrIODRA  511695057-95325C34532B3A7F201FADE52D993D60   
9   14Yppg7V9X8gfJn1fSuimm  122707841-789A410EEA9ED723DAF215C452F513D4   
10  1bzr2d23lvzACpFcNRg7dd  515565972-49CC18ED21F2CC53EE91A34B8C001DF9   
11  1oi4Iwkv9MN65QmQVYGnaW    3674600-8EB239261482267